In [1]:
import pandas as pd
import math
from statistics import mode 
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
import math
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

In [2]:
class NaiveBayes(GaussianNB):
    trainDict=None
    outCol=None
    y_train=None
    X_train=None
    gausscianNaiveBayes=None
    predictions=None
    TestInd=None
    
    def CategTraining(self,X_train,y_train,outCol):
        trainCount=y_train[outCol[0]].value_counts().to_dict()
        col1=list(X_train.columns)
        outputList=y_train[outCol[0]].value_counts().keys().tolist()
        trainDict=dict([(key, []) for key in col1])
        for i in range(0,len(col1)):
            inputList=X_train[col1[i]].value_counts().keys().tolist()
            proxy=dict([(key, dict([(keys, []) for keys in outputList])) for key in inputList])
            trainDict[col1[i]]=proxy
        for i in trainDict.keys():
            for j in trainDict[i].keys():
                for k in trainDict[i][j].keys():
                    num=(X_train.loc[(X_train[i]==j) &  (y_train[outCol[0]]==k)]).shape[0]
                    den=trainCount[k]
                    prob=num/den
                    trainDict[i][j][k]=prob
        self.trainDict=trainDict
        self.outCol=outCol
        self.y_train=y_train
        self.X_train=X_train
    
    def CategTesting(self,X_test):
        y_train=self.y_train
        outCol=self.outCol
        trainDict=self.trainDict
        trainProb=(y_train[outCol[0]].value_counts()/y_train.shape[0]).to_dict()
        outputList=y_train[outCol[0]].value_counts().keys().tolist()
        testDict1=dict([(key,dict([(keys, []) for keys in outputList])) for key in X_test.index])
        for i in testDict1.keys():
            for j in testDict1[i].keys():
                prob=1
                l=0
                for k in trainDict.keys():
                    prob=trainDict[k][X_test.loc[i][l]][j]*prob
                    l=l+1
                testDict1[i][j]=prob*trainProb[j]
        return testDict1
    
    
    def fit(self, X, y):
        X_cont=X.select_dtypes(exclude='object')
        self.gausscianNaiveBayes=super().fit( X_cont, y)
        outCol=y.columns.tolist()
        X_Categ=X.select_dtypes(include='object')
        self.CategTraining(X_Categ,y,outCol)
        return self
    
    def predict(self,X):
        X_Categ=X.select_dtypes(include='object')
        CategtestDict=self.CategTesting(X_Categ)
        X_Cont=X.select_dtypes(exclude='object')
        outCol=self.outCol
        ContpredictProb=self.gausscianNaiveBayes.predict_proba(X_Cont)
        y_train=self.y_train
        posOp=y_train[outCol[0]].value_counts().keys().tolist()
        testInd=X.index.tolist()
        FinalPrediction=[]
        r=0
        c=0
        ContTestDict=dict([(key,dict([(keys, []) for keys in posOp])) for key in X.index])
        for i in ContTestDict.keys():
            c=0
            for j in ContTestDict[i].keys():
                ContTestDict[i][j]=ContpredictProb[r][c]
                c=c+1
            r=r+1
        size=X.shape[0]
        for i in ContTestDict.keys():
            maxi=0
            pr=''
            p=0
            for j in posOp:
                p=ContTestDict[i][j]*CategtestDict[i][j]
                if(p>maxi):
                    maxi=p
                    pr=j
            FinalPrediction.append(pr)
        return FinalPrediction
    
    def score(self, X, y):
        X_Categ=X.select_dtypes(include='object')
        CategtestDict=self.CategTesting(X_Categ)
        X_Cont=X.select_dtypes(exclude='object')
        outCol=self.outCol
        ContpredictProb=self.gausscianNaiveBayes.predict_proba(X_Cont)
        y_train=self.y_train
        posOp=y_train[outCol[0]].value_counts().keys().tolist()
        testInd=X.index.tolist()
        FinalPrediction=[]
        r=0
        c=0
        ContTestDict=dict([(key,dict([(keys, []) for keys in posOp])) for key in X.index])
        for i in ContTestDict.keys():
            c=0
            for j in ContTestDict[i].keys():
                ContTestDict[i][j]=ContpredictProb[r][c]
                c=c+1
            r=r+1
        size=X.shape[0]
        for i in ContTestDict.keys():
            maxi=0
            pr=''
            p=0
            for j in posOp:
                p=ContTestDict[i][j]*CategtestDict[i][j]
                if(p>maxi):
                    maxi=p
                    pr=j
            FinalPrediction.append(pr)
        from sklearn import metrics
        return metrics.accuracy_score(y,FinalPrediction)   

In [53]:
class Encoding(OneHotEncoder):
    one=None
    data=None
    CategCol=None
    def fit(self, X):
        CategX=X.select_dtypes(include='object')
        self.CategCol=CategX.columns.tolist()
        one=OneHotEncoder(handle_unknown='ignore')
        self.one=one.fit(CategX)
        return one.fit(CategX)
    
    def transform(self,X):
        ContX=X.select_dtypes(exclude='object')
        CategX=X.select_dtypes(include='object')
        binaryValues=self.one.transform(CategX).toarray()
        newCol=list(self.one.get_feature_names(self.CategCol))
        return binaryValues
        

In [54]:
e=Encoding()
e.fit(X)
e.transform(X)

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [3]:
def removeQuestionMark(X,y):
    outCol=y.columns.tolist()
    inpCol=X.columns.tolist()
    for i in inpCol:
        repl=X[i].value_counts().keys().tolist()[0]
        X[i]=X[i].replace(to_replace=' ?',value=repl)
    rep=y[outCol[0]].value_counts().keys().tolist()[0]
    y[outCol[0]]=y[outCol[0]].replace(to_replace=' ?',value=rep)
    return X,y
    
def Splitting(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    return [X_train,y_train,X_test,y_test]


In [4]:
df=pd.read_csv("C:\\Users\\Trivikram\\ML_Projects\\Data\\adult.data",header=None,names=['Age','Workplace','fnlwgt','education','education num','marital-stauts','occupation','relationship','race','sex','capital gain','capital loss','hours per week','native country','Salary'])
df

,Age,Workplace,fnlwgt,education,education num,marital-stauts,occupation,relationship,race,sex,capital gain,capital loss,hours per week,native country,Salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [5]:
X=df[[col for col in df.columns if col!='Salary']]
y=df[['Salary']]
X,y=removeQuestionMark(X,y)
X_train,y_train,X_test,y_test=Splitting(X,y)
n1=NaiveBayes()
n1.fit(X_train,y_train)
n1.score(X_test,y_test)

C:\Users\Trivikram\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Trivikram\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Trivikram\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8195311700276384

In [30]:
pipe = Pipeline(steps=[('encoding',None),('scl',None),('classifier',NaiveBayes())])
search_space = [{'encoding':[OneHotEncoder(handle_unknown='ignore')],'scl':[ StandardScaler()],'classifier':[KNeighborsClassifier()],
                'classifier__n_neighbors':range(2,20)}]

In [33]:
clf = GridSearchCV(pipe,search_space, cv=5, verbose=0)
clf.fit(a,y)

C:\Users\Trivikram\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:530: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


ValueError: Cannot center sparse matrices: pass `with_mean=False` instead. See docstring for motivation and alternatives.

In [32]:
a=X.select_dtypes(exclude='object')

In [51]:
neight = KNeighborsClassifier()
length=int(math.sqrt(X_test.shape[0]))
parameters={'n_neighbors':([i for i in range(2,20)])}
clf = GridSearchCV(neight, parameters)

In [55]:
pipe_gnb = Pipeline([('NB', NaiveBayes())])
pipelines =  [pipe_gnb]
pipe_dict = {0: 'Naive Bayes'}
for pipe in pipelines:
    pipe.fit(X_train, y_train)
for idx, val in enumerate(pipelines):
    print('%s pipeline test accuracy: %.3f' % (pipe_dict[idx], val.score(X_test, y_test)))

C:\Users\Trivikram\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Gausscian Naive Bayes pipeline test accuracy: 0.820


In [14]:
# Create a pipeline
pipe = Pipeline(steps=[('encoding',None),('scl',None),('classifier',NaiveBayes())])

# Create space of candidate learning algorithms and their hyperparameters
search_space = [{'classifier':[KNeighborsClassifier()],'scl':[ StandardScaler()],'encoding':[OneHotEncoder(handle_unknown='ignore')],
                'classifier__n_neighbors':range(2,20)}]


In [21]:
clf = GridSearchCV(pipe, search_space, cv=5, verbose=0)
a=X.select_dtypes(exclude='object')
a

,Age,fnlwgt,education num,capital gain,capital loss,hours per week
0,39,77516,13,2174,0,40
1,50,83311,13,0,0,13
2,38,215646,9,0,0,40
3,53,234721,7,0,0,40
4,28,338409,13,0,0,40
...,...,...,...,...,...,...
32556,27,257302,12,0,0,38
32557,40,154374,9,0,0,40
32558,58,151910,9,0,0,40
32559,22,201490,9,0,0,20


In [27]:
pipe.fit(X,y)
pipe.score(X,y)

C:\Users\Trivikram\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8149933970086913

In [ ]:
class